<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_003_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Efecto del tamaño de la muestra en la validación cruzada</font>**

<p align="justify">
👀 En el Colab anterior, presentamos el marco general de validación cruzada y cómo evaluar si un modelo predictivo está simplificado, sobreajustando o generalizando. Además de estos aspectos, también es importante entender cómo los diferentes errores están influenciados por el número de muestras disponibles.
<br><br>
En este Colab, mostraremos este aspecto al observar la variabilidad de los diferentes errores. Vamos a cargar los datos y crear el mismo modelo que en el Colab anterior.

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

 ## **<font color="DeepPink">Carga y análisis de conjunto de datos</font>**

<p align = "justify">
👀 Vamos a usar datos de <code>Scikit-learn</code>

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
housing = fetch_california_housing(as_frame=True) #as_frame: como Pandas DataFrame

In [ ]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

<p align="justify">
👀 En este conjunto de datos, el objetivo es predecir el valor promedio de las casas en un área de California. Las características recopiladas se basan en información inmobiliaria y geográfica general de estas propiedades. Como el valor promedio a predecir es el precio medio (variable numérica continua), utilizaremos entonces un modelo predictivo específico de regresión. 

In [ ]:
housing.data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


<p align="justify">
👀 Ahora si, dividimos todos los datos del conjunto de datos <code>housing</code>, en nuestra variable objetivo y las variables explicativas. En el caso de la variable objetivo hacemos una transformación de los datos para expresar los valores a cientos de miles de dólares. 

In [ ]:
X, y = housing.data, housing.target
y *= 100000

 # **<font color="DeepPink">Curva de aprendizaje</font>**


<p align = "justify">
Para comprender el impacto del número de muestras disponibles para el entrenamiento en el rendimiento de generalización de un modelo predictivo, es posible reducir sintéticamente el número de muestras utilizadas para entrenar el modelo predictivo y verificar los errores de entrenamiento y prueba.
<br><br>
Por lo tanto, podemos variar el número de muestras en el conjunto de entrenamiento y repetir el experimento. Los puntajes de entrenamiento y prueba se pueden trazar de manera similar a la curva de validación, pero en lugar de variar un hiperparámetro, variamos la cantidad de muestras de entrenamiento.
<br><br>
Esta curva se llama la curva de aprendizaje.
<br><br>
Brinda información sobre el beneficio de agregar nuevas muestras de entrenamiento para mejorar el rendimiento de generalización de un modelo.
Calculemos la curva de aprendizaje para un árbol y varíemos la proporción del conjunto de entrenamiento del 10% al 100%.

In [ ]:
train_sizes = np.linspace(0.1, 1.0, num=5, endpoint=True)
train_sizes

array([0.1  , 0.325, 0.55 , 0.775, 1.   ])

 ## **<font color="DeepPink">Creación del modelo</font>**

Para resolver el problema de regresión, utilizaremos un [Árbol de Regresión](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(random_state=0) 
model

DecisionTreeRegressor(random_state=0)

In [ ]:
from sklearn.model_selection import ShuffleSplit

In [ ]:
cv = ShuffleSplit(n_splits=30, test_size=0.2)

<p align="justify">
👀 Ahora llevamos a cabo el experimento. 

 ## **<font color="DeepPink">Curva de aprendizaje</font>**

In [ ]:
from sklearn.model_selection import learning_curve

In [ ]:
results = learning_curve(model, 
                         X, y, 
                         train_sizes=train_sizes, 
                         cv=cv,
                         scoring="neg_mean_absolute_error", 
                         n_jobs=2)

In [ ]:
train_size, train_scores, test_scores = results[:3]

In [ ]:
train_errors, test_errors = -train_scores, -test_scores

<p align="justify">
📊 Ahora que recopilamos los resultados, mostramos la curva de aprendizaje.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=train_sizes, 
                         y=train_errors.mean(axis=1), 
                         name='Training error'))

fig.add_trace(go.Scatter(x=train_sizes, 
                         y=test_errors.mean(axis=1), 
                         name='Testing error'))

fig.update_layout(template="gridon",
                  title_text="Learning curve for Decision Tree",
                  xaxis_title='Number of samples in the training set',
                  yaxis_title='Mean Absolute Error (k$)')

fig.show()

<p align="justify">
👀 Mirando solo el error de entrenamiento, vemos que obtenemos un error de 0 k$. Significa que el modelo entrenado (es decir, el árbol) está claramente sobreajustado a los datos de entrenamiento.
<br><br>
Mirando solo el error de prueba, observamos que mientras más muestras se agreguen al conjunto de entrenamiento, menor será el error de prueba. Además, estamos buscando la meseta del error de prueba para el cual ya no hay beneficio en agregar muestras o evaluar la ganancia potencial de agregar más muestras en el conjunto de entrenamiento.
<br><br>
Si logramos una meseta y al agregar nuevas muestras en el conjunto de entrenamiento no reduce el error de prueba, es posible que hayamos alcanzado la tasa de error de <code>Bayes</code> utilizando el modelo disponible. El uso de un modelo más complejo podría ser la única posibilidad de reducir aún más el error de prueba.



 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un objeto de<code>Scikit-learn</code> y lo convertimos a un <code>DataFrame</code>.
<br> 
✅ Verificamos la influencia del número de muestras en un conjunto de datos, especialmente en la variabilidad de los errores informados al ejecutar la validación cruzada.
<br> 
✅ Aprendimos sobre la curva de aprendizaje, que es una representación visual de la capacidad de un modelo para mejorar añadiendo nuevas muestras. 

<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
